<a href="https://colab.research.google.com/github/TillVollmer5/mass_spectroscopy/blob/main/Benzylbenzoate_recovery_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Benzylbenzoate Recovery Analysis
This notebook aims at investigating the recovery of the internal standard in the water extraction with n-hexan and a mixture of n-hexan and 5% diethyl ether. This is just a short script to quickly asses the recovery. As the equilibrium state of the benzylbenzoate between the organic phase and the liquid phase leads to a decrease in the signal of the chromatogram and falsifies the concentration calculation of the phthalates.

All the input files are added to a google drive folder with the name *Benzylbenzoate_recovery_input*.



In [22]:
from google.colab import drive
import glob
import pandas as pd
from datetime import datetime
import csv
import numpy as np
from scipy import stats

drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [3]:
unextract_files_list = glob.glob(f'/content/drive/My Drive/Benzylbenzoate_recovery_input/BB_recov_*')
unextract_df = pd.DataFrame()

for unextract_file_path in unextract_files_list:
  unextract_data = pd.read_csv(unextract_file_path, header=4)
  unextract_df = unextract_df.append(unextract_data)


unextract_df = unextract_df.reset_index(drop=True)

print(unextract_df)

<ipython-input-3-fdb51e625da6>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unextract_df = unextract_df.append(unextract_data)
<ipython-input-3-fdb51e625da6>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unextract_df = unextract_df.append(unextract_data)
<ipython-input-3-fdb51e625da6>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unextract_df = unextract_df.append(unextract_data)
<ipython-input-3-fdb51e625da6>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unextract_df = unextract_df.append(unextract_data)
<ipython-input-3-fdb51e625da6>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas i

   Apex RT  Start RT  End RT          Area  %Area        Height  %Height
0    20.80     20.74   20.85  2.173541e+10    100  9.581892e+09      100
1    20.80     20.75   20.85  2.364346e+10    100  1.053549e+10      100
2    20.80     20.75   20.85  1.747253e+10    100  8.171063e+09      100
3    20.81     20.75   20.89  2.251730e+10    100  1.017443e+10      100
4    20.80     20.75   20.86  1.545550e+10    100  7.170586e+09      100
5    20.80     20.75   20.86  2.002994e+10    100  9.492307e+09      100


<ipython-input-3-fdb51e625da6>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unextract_df = unextract_df.append(unextract_data)


In [6]:
extract_files_list = glob.glob(f'/content/drive/My Drive/Benzylbenzoate_recovery_input/Spike_water_*')
extract_df = pd.DataFrame()

for extract_file_path in extract_files_list:
  extract_data = pd.read_csv(extract_file_path, header=4)
  mask = extract_data['Apex RT'].isin([20.80, 20.79, 20.81])
  extract_rows = extract_data[mask]
  extract_df = extract_df.append(extract_rows)


extract_df = extract_df.reset_index(drop=True)

print(extract_df)

   Apex RT  Start RT  End RT          Area  %Area        Height  %Height
0    20.79     20.73   20.86  1.285649e+10  32.35  6.025451e+09    31.87
1    20.80     20.76   20.84  2.529787e+10  75.76  1.154138e+10    76.42
2    20.80     20.75   20.86  1.827266e+10  33.19  8.291809e+09    32.44
3    20.81     20.75   20.87  3.612363e+10  78.02  1.529648e+10    77.04
4    20.80     20.75   20.88  1.174459e+10  35.51  5.587832e+09    35.40
5    20.81     20.75   20.86  2.616546e+10  81.98  1.198183e+10    82.53
6    20.80     20.75   20.89  1.137054e+10  36.92  5.423433e+09    36.52
7    20.81     20.75   20.90  2.613526e+10  82.41  1.220199e+10    83.46


<ipython-input-6-d1acfae7705d>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extract_df = extract_df.append(extract_rows)
<ipython-input-6-d1acfae7705d>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extract_df = extract_df.append(extract_rows)
<ipython-input-6-d1acfae7705d>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extract_df = extract_df.append(extract_rows)
<ipython-input-6-d1acfae7705d>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extract_df = extract_df.append(extract_rows)
<ipython-input-6-d1acfae7705d>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use 

In [31]:
unextract_mean = unextract_df.mean()
extract_mean = extract_df.mean()
unextract_sem = unextract_df.sem()
extract_sem = extract_df.sem()

recov =(extract_mean / unextract_mean) * 100

error_recov = np.sqrt((((-extract_mean / unextract_mean**2) * unextract_sem)**2) + ((1 / unextract_mean) * extract_sem)**2)

print(unextract_mean)
print(extract_mean)
print(unextract_sem)
print(extract_sem)
print(recov)
print(error_recov )

Apex RT     2.080167e+01
Start RT    2.074833e+01
End RT      2.086000e+01
Area        2.014236e+10
%Area       1.000000e+02
Height      9.187629e+09
%Height     1.000000e+02
dtype: float64
Apex RT     2.080250e+01
Start RT    2.074875e+01
End RT      2.087000e+01
Area        2.099581e+10
%Area       5.701750e+01
Height      9.543774e+09
%Height     5.696000e+01
dtype: float64
Apex RT     1.666667e-03
Start RT    1.666667e-03
End RT      6.324555e-03
Area        1.285082e+09
%Area       0.000000e+00
Height      5.209180e+08
%Height     0.000000e+00
dtype: float64
Apex RT     2.500000e-03
Start RT    2.950484e-03
End RT      6.813851e-03
Area        3.140726e+09
%Area       8.559674e+00
Height      1.313847e+09
%Height     8.713022e+00
dtype: float64
Apex RT     100.004006
Start RT    100.002008
End RT      100.047939
Area        104.237111
%Area        57.017500
Height      103.876356
%Height      56.960000
dtype: float64
Apex RT     0.000144
Start RT    0.000163
End RT      0.000446
A

In [32]:
print(unextract_mean)
print(extract_mean)
print(recov)

current_date = datetime.today().strftime('%Y-%m-%d')
row_descriptors = ['Extracted values', 'Unextracted value', 'Recovery [%]']
header_row1 = ['', 'Values', 'Error', 'Error%']
header_row0 = ['Recovery analysis of Benzylbenzoate:', current_date, 'Till Vollmer']
output_dir = '/content/drive/My Drive/Benzylbenzoate_recovery_output/Recovery_analysis_Benzylbenzoate_Data.csv'

error_per_e = extract_sem['Area'] / extract_mean['Area']
error_per_u = unextract_sem['Area'] / unextract_mean['Area']

row_data1 = [row_descriptors[0], extract_mean['Area'], extract_sem['Area'], error_per_e]
row_data2 = [row_descriptors[1], unextract_mean['Area'], unextract_sem['Area'], error_per_u]
row_data3 = [row_descriptors[2], recov['Area'], error_recov['Area']]

with open(output_dir, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header_row0)
    writer.writerow(header_row1)
    writer.writerow(row_data1)
    writer.writerow(row_data2)
    writer.writerow(row_data3)

Apex RT     2.080167e+01
Start RT    2.074833e+01
End RT      2.086000e+01
Area        2.014236e+10
%Area       1.000000e+02
Height      9.187629e+09
%Height     1.000000e+02
dtype: float64
Apex RT     2.080250e+01
Start RT    2.074875e+01
End RT      2.087000e+01
Area        2.099581e+10
%Area       5.701750e+01
Height      9.543774e+09
%Height     5.696000e+01
dtype: float64
Apex RT     100.004006
Start RT    100.002008
End RT      100.047939
Area        104.237111
%Area        57.017500
Height      103.876356
%Height      56.960000
dtype: float64
